# Librerias y dependencias

In [15]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import folium
from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster
import imageio
import tempfile
from sklearn.metrics import mean_absolute_percentage_error

# Entrenamiento

## Preparativos

### Parametros

In [ ]:
DATOS_PATH = r"Datasets\Datos_de_entrenamiento.csv"
DIR_PATH = r"Datasets\Directorio_municipios_procesado.csv"
OUTPUT_DIR = r"output_prophet_maps"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# Horizon de predicción (hasta diciembre 2025)
PRED_END = "2025-12-31"


# Selección: si deseas entrenar todos los municipios pon None, si prefieres top_k por visitas totales pon un entero
# TOP_K = 10
TOP_K = None
MIN_MONTHS_FOR_MODEL = 6 # mínimo meses de datos para entrenar

### Funciones auxiliares

In [22]:
def prepare_prophet_df(df, municipio):
    d = df[df['MUNICIPIO'] == municipio][['ds','VISITAS']].rename(columns={'VISITAS':'y'}).sort_values('ds')
    # llenar meses faltantes para continuidad (freq = MS)
    if d.empty:
        return d
    idx = pd.date_range(d['ds'].min(), d['ds'].max(), freq='MS')
    d = d.set_index('ds').reindex(idx).rename_axis('ds').reset_index()
    d['y'] = d['y'].fillna(0)
    return d

def train_and_forecast(df_ts, periods_months):
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    # añadir seasonality mensual si se quiere:
    m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    m.fit(df_ts)
    future = m.make_future_dataframe(periods=periods_months, freq='MS')
    forecast = m.predict(future)
    return m, forecast

def months_between_dates(start, end):
    s = pd.to_datetime(start)
    e = pd.to_datetime(end)
    return (e.year - s.year) * 12 + (e.month - s.month)

## Carga y procesamiento

In [ ]:
# Carga y preprocesamiento básico
df_raw = pd.read_csv(DATOS_PATH)
df_dir = pd.read_csv(DIR_PATH)


# Esperamos: df_raw con MES,AÑO,MUNICIPIO,VISITAS
# Crear columna fecha (usaremos primer día del mes)
df_raw['MES'] = df_raw['MES'].astype(int)
df_raw['AÑO'] = df_raw['AÑO'].astype(int)
df_raw['DIA'] = pd.to_datetime(df_raw['AÑO'].astype(str) + '-' + df_raw['MES'].astype(str).str.zfill(2) + '-01')


# Merge para unir coordenadas (puede haber municipios sin coordenadas)
df_merge = df_raw.merge(df_dir[['MUNICIPIO','LONGITUD','LATITUD']], on='MUNICIPIO', how='left')


agg_total = df_merge.groupby('MUNICIPIO', as_index=False)['VISITAS'].sum().sort_values('VISITAS', ascending=False)
if TOP_K is None:
    lstMunicipios = agg_total['MUNICIPIO'].tolist()
else:
    lstMunicipios = agg_total.head(TOP_K)['MUNICIPIO'].tolist()

print(f"Cantidad de municipios seleccionados: {len(lstMunicipios)}")
print(f"Municipios seleccionados: {lstMunicipios}")



# Preparar horizonte en meses desde último dato hasta PRED_END
last_date = df_merge['DIA'].max()
print('Último dato en:', last_date)
periods_to_predict = months_between_dates(last_date, PRED_END)
print('Meses a predecir:', periods_to_predict)

# Entrenamiento

In [ ]:
results = {}
for municipio in lstMunicipios:
    df_ts = prepare_prophet_df(df_merge, municipio)
    if df_ts.shape[0] < MIN_MONTHS_FOR_MODEL:
        print(f"Omitido {municipio}: solo {df_ts.shape[0]} meses de datos")
        continue
    try:
        model, forecast = train_and_forecast(df_ts, periods_to_predict)
        results[municipio] = {'model': model, 'forecast': forecast, 'history': df_ts}
        # Guardar gráfica simple
        fig = model.plot(forecast)
        fig.savefig(os.path.join(OUTPUT_DIR, municipio, f'forecast.png'), bbox_inches='tight')
        plt.close(fig)
        comp = model.plot_components(forecast)
        comp.savefig(os.path.join(OUTPUT_DIR, f'components.png'), bbox_inches='tight')
        plt.close(comp)
        print(f"Modelo generado para {municipio}")
    except Exception as e:
        print('Error en ', municipio, e)

22:46:39 - cmdstanpy - INFO - Chain [1] start processing
22:46:41 - cmdstanpy - INFO - Chain [1] done processing
22:46:41 - cmdstanpy - INFO - Chain [1] done processing
22:46:41 - cmdstanpy - INFO - Chain [1] start processing
22:46:41 - cmdstanpy - INFO - Chain [1] start processing


Modelo generado para PAIPA


22:46:52 - cmdstanpy - INFO - Chain [1] done processing
22:46:52 - cmdstanpy - INFO - Chain [1] start processing
22:46:52 - cmdstanpy - INFO - Chain [1] start processing


Modelo generado para VILLA DE LEYVA


22:47:02 - cmdstanpy - INFO - Chain [1] done processing
22:47:02 - cmdstanpy - INFO - Chain [1] start processing
22:47:02 - cmdstanpy - INFO - Chain [1] start processing


...

Modelo generado para PUERTO BOYACA
Omitido PACHAVITA: solo 2 meses de datos
Omitido UMBITA: solo 2 meses de datos


22:50:34 - cmdstanpy - INFO - Chain [1] done processing
22:50:34 - cmdstanpy - INFO - Chain [1] start processing
22:50:34 - cmdstanpy - INFO - Chain [1] start processing


Modelo generado para ZETAQUIRA


22:50:34 - cmdstanpy - INFO - Chain [1] done processing


Modelo generado para TASCO
Omitido TINJACA: solo 5 meses de datos
Omitido TUNUNGUA: solo 3 meses de datos
Omitido GUACAMAYAS: solo 1 meses de datos
Omitido PAUNA: solo 1 meses de datos
Omitido MUZO: solo 1 meses de datos
Omitido LA UVITA: solo 1 meses de datos
Omitido SAN EDUARDO: solo 1 meses de datos
Omitido SAN JOSE DE PARE: solo 1 meses de datos
Omitido SANTANA: solo 1 meses de datos
Omitido SATIVANORTE: solo 1 meses de datos
Omitido SUSACON: solo 1 meses de datos
Omitido TIBANA: solo 1 meses de datos
Omitido TOGUI: solo 1 meses de datos
Omitido TUTAZA: solo 1 meses de datos


# Evaluacion

In [27]:
# Evaluación rápida (MAPE) sobre el período observado usando los puntos de backtest en forecast
evals = []
for muni, obj in results.items():
    hist = obj['history']
    f = obj['forecast']
    # tomar solo las fechas que estaban en el histórico
    f_obs = f[f['ds'].isin(hist['ds'])]
    if f_obs.empty:
        continue
    y_true = hist['y'].values
    y_pred = f_obs['yhat'].values
    mape = mean_absolute_percentage_error(y_true, y_pred)
    evals.append({'MUNICIPIO': muni, 'MAPE': mape})


pd.DataFrame(evals).sort_values('MAPE').to_csv(os.path.join(OUTPUT_DIR, 'evaluation_mape.csv'), index=False)

## -----------------------------
## Preparacion datos mensuales por municipio

In [28]:
# Preparar data mensual por municipio para el mapa (unir histórico + predicción)
# Vamos a construir una tabla con columnas: ds, MUNICIPIO, VISITAS (observado o predicho), LONGITUD, LATITUD
map_rows = []
for muni, obj in results.items():
    f = obj['forecast'][['ds','yhat']].copy().rename(columns={'yhat':'VISITAS'})
    # si quieres marcar qué es observado vs predicho:
    last_hist = obj['history']['ds'].max()
    f['TIPO'] = np.where(f['ds'] <= last_hist, 'OBS', 'PRED')
    # adjuntar coordenadas
    coords_df = df_dir[df_dir['MUNICIPIO']==muni][['LONGITUD','LATITUD']]
    if not coords_df.empty:
        coords = coords_df.iloc[0].to_dict()
        f['LONGITUD'] = coords.get('LONGITUD', None)
        f['LATITUD'] = coords.get('LATITUD', None)
    else:
        f['LONGITUD'] = np.nan
        f['LATITUD'] = np.nan
    f['MUNICIPIO'] = muni
    map_rows.append(f)


map_df = pd.concat(map_rows, ignore_index=True)
# Filtrar filas sin coords
map_df = map_df.dropna(subset=['LONGITUD','LATITUD'])


# Normalizar intensidad por mes (opcional) para que colores sean comparables
# Opción A: normalizar por valor máximo global
max_global = map_df['VISITAS'].max()
map_df['INTENSIDAD_GLOB'] = map_df['VISITAS'] / (max_global if max_global>0 else 1)
# Opción B: normalizar por mes (si prefieres comparar distribución interna de cada mes)
map_df['MES_STR'] = map_df['ds'].dt.strftime('%Y-%m')
map_df['INTENSIDAD_MES'] = map_df.groupby('MES_STR')['VISITAS'].transform(lambda x: x / (x.max() if x.max()>0 else 1))


map_df.to_csv(os.path.join(OUTPUT_DIR,'map_df.csv'), index=False)

## Visualizacion con mapa

In [29]:
# Crear un mapa con HeatMapWithTime + MarkerCluster. HeatMapWithTime espera una lista de matrices de [lat,lon,intensity]
# Preparar lista ordenada de meses
months = sorted(map_df['MES_STR'].unique())
heat_data = []
for m in months:
    sub = map_df[map_df['MES_STR']==m]
    # HeatMap expects [lat, lon, weight]
    heat_data.append(sub[['LATITUD','LONGITUD','INTENSIDAD_GLOB']].values.tolist())


mid_lat = map_df['LATITUD'].mean()
mid_lon = map_df['LONGITUD'].mean()


m = folium.Map(location=[mid_lat, mid_lon], zoom_start=6)


# HeatMapWithTime
HeatMapWithTime(heat_data, index=months, auto_play=False, max_opacity=0.8).add_to(m)


# MarkerCluster para mostrar puntos (puede saturar la vista si hay muchos)
mc = MarkerCluster()
for _, row in map_df[map_df['ds'] == map_df['ds'].min()].iterrows():
    folium.Marker(location=[row['LATITUD'], row['LONGITUD']], popup=f"{row['MUNICIPIO']}").add_to(mc)
mc.add_to(m)


# Guardar mapa HTML
map_html_path = os.path.join(OUTPUT_DIR, 'heatmap_time.html')
m.save(map_html_path)
print('Mapa con time saved en:', map_html_path)

Mapa con time saved en: output_prophet_maps\heatmap_time.html


## Generacion Gif

In [ ]:
# Generar GIF: estrategia
# 1) Crear snapshots de mapas por mes (renderizar HTML a PNG). Para ello puedes usar selenium + chromium headless.
# 2) Convertir PNGs a GIF con imageio.


# Código de ejemplo (requiere selenium y un driver apropiado):
try:
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    # Ajusta path del chromedriver si es necesario
    driver = webdriver.Chrome(options=chrome_options)
    images = []
    tmp_dir = tempfile.mkdtemp()
    for mth in months:
        # construir un mapa estático por mes (heatmap con peso solo ese mes)
        sub = map_df[map_df['MES_STR']==mth]
        mf = folium.Map(location=[mid_lat, mid_lon], zoom_start=6)
        HeatMap(sub[['LATITUD','LONGITUD','INTENSIDAD_GLOB']].values.tolist(), radius=25, max_zoom=13).add_to(mf)
        fn = os.path.join(tmp_dir, f'map_{mth}.html')
        outpng = os.path.join(OUTPUT_DIR, f'map_{mth}.png')
        mf.save(fn)
        driver.set_window_size(1200, 800)
        driver.get('file://' + os.path.abspath(fn))
        # esperar un poco si tu máquina es lenta
        driver.implicitly_wait(1)
        driver.save_screenshot(outpng)
        images.append(imageio.imread(outpng))
    driver.quit()
    gif_path = os.path.join(OUTPUT_DIR, 'timelapse_maps.gif')
    imageio.mimsave(gif_path, images, duration=0.7)
    print('GIF guardado en:', gif_path)
except Exception as e:
    print('No se generó GIF automáticamente. Error o falta selenium/chromedriver:', e)
    print('Alternativa: abrir', map_html_path, 'y usar una herramienta para grabar o exportar frames manually')